# Event Monitor

## Monitor Structure

Description here

Event Monitor
* start_monitor
* stop_monitor
* get_monitor_status
* monitor

Event Sync
* start_sync
* stop_sync
* watch_sync
* get_sync_status
* sync

Event Handler
* start_event_handler
* stop_event_handler
* get_event_handler_status
* event_handler
* handle_event

Helpers
* get_events_df
* get_events_list
* get_queue_size
* get_contract_events

## Setup

The event monitor requires a blockexplorer for initialization. The purpose of this blockexplorer is to get the contract source code so that contract interactions (Data & Logs) can be decoded into human readable events. 

In [14]:
from messari.blockexplorers import Etherscan

API_KEY = 'YOUR_API_KEY'
ES = Etherscan(api_key=API_KEY)

The event monitor also requires the endpoint for an RPC. This should belong to the same network as the blockexplorer.

In [15]:
from messari.eventmonitor import EventMonitor

rpc_url = 'YOUR_RPC'

Finally the event monitor must be initialized with the adress to monitor for events. 

NOTE: The source code for this contract must be verified on the selected blockexplorer or else the event monitor will not be able to decode events for the given contract

In [12]:
DAI = '0x6B175474E89094C44Da98b954EedeAC495271d0F'

Initalize with contract(s), blockexplorer, & RPC

In [13]:
dai_monitor = EventMonitor(DAI, ES, rpc_url)

## Event Monitor

### start_monitor

Start the event monitor in a dedicated thread. The monitor thread will continue to run until `stop_monitor()` is called. The monitor thread can be accessed with `EventMonitor.monitor_thread`.

In [5]:
dai_monitor.start_monitor()

### stop_monitor

Stop the event monitor thread. This function will set an internal flag in the `EventMonitor` which tells the monitor thread to exit its loop. The monitor thread will not stop until it finishes the current loop and reaches the top again.

In [9]:
dai_monitor.stop_monitor()

### get_monitor_status

Returns the operating status of the event monitor
- RUNNING: event monitor is running
- STOPPING: event monitor is stopping. If the monitor takes too long to stop then something is likely wrong
- CRASH: event monitor thread crashed.
- DEAD: event monitor is dead/stopped. 

In [8]:
monitor_status = dai_monitor.get_monitor_status()
monitor_status

'RUNNING'

### monitor

This function runs the main loop for live event monitoring

WARNING: It is highly recomended that you only interact with this function through the `start_monitor()` and `stop_monitor()` functions, not directly. However if you are inclined, feel free to read the source code for the EventMonitor class and experiment with this function in your own flow/script.

In [7]:
# dai_monitor.monitor()

## Event Sync

### start_sync

Start the event sync in a dedicated thread. The sync thread will find and submit all events in the given block range to the event handler. The sync thread will continue to run until `stop_sync()` is called or the sync is finished. The sync thread can be accessed with `self.sync_thread`.

The parameters for this function are:
* start: starting block, default is 0
* end: ending block, default is 'latest'

In [12]:
dai_monitor.start_sync()

### stop_sync

Stop the event sync thread. This function will set an internal flag in the `EventMonitor` which tells the sync thread to exit its loop. The sync thread will not stop until it finishes the current loop and reaches the top again.

In [15]:
dai_monitor.stop_sync()

### watch_sync

This function can be used to watch the status of any given sync. Every 10 seconds this function will print an update on the progress of a sync.

NOTE: This is a a blocking function meaning it will be run on the main thread until concluding. ie if there is 10 minutes left in a sync and you call this function, you'll be watching this function for the next 10 minutes

In [21]:
# dai_monitor.watch_sync()

### get_sync_status

Returns the operating status of the event sync
- RUNNING: event sync is running
- STOPPING: event sync is stopping. If the sync takes too long to stop then something is likely wrong
- CRASH: event sync thread crashed.
- DEAD: event sync is dead/stopped. 

In [ ]:
sync_status = dai_monitor.get_sync_status()
sync_status

### sync

This function runs the main loop for syncing

WARNING: It is highly recomended that you only interact with this function through the `start_sync()` and `stop_sync()` functions, not directly. However if you are inclined, feel free to read the source code for the EventMonitor class and experiment with this function in your own flow/script.

In [ ]:
# dai_monitor.sync(0, 100)

## Event Handler

### start_event_handler

Start the event handler in a dedicated thread. The handler thread will continue to run until `stop_handler()` is called. The handler thread can be accessed with `self.handler_thread`.

NOTE: this is automatically called with the `start_monitor()` and `start_sync()` functions. You should not have to call this function

In [ ]:
dai_monitor.start_event_handler()

### stop_event_handler

Stop the event handler thread. This function will set an internal flag in the `EventMonitor` which tells the handler thread to exit its loop. The sync thread will not stop until it finishes the current loop and reaches the top again.

NOTE: The event handler thread will run until all events have been handled, you should be able to let this thread run until completion

In [21]:
dai_monitor.stop_event_handler()

### get_event_handler_status

Returns the operating status of the event handler

- RUNNING: event handler is running
- STOPPING: event handler is stopping. If the handler takes too long to stop then something is likely wrong
- CRASH: event handler thread crashed.
- DEAD: event handler is dead/stopped. 

In [11]:
event_handler_status = dai_monitor.get_event_handler_status()
event_handler_status

'DEAD'

### event_handler

This function runs the main loop for event handling

WARNING: It is highly recomended that you only interact with this function through the `start_event_handler()` and `stop_event_handler()` functions, not directly. However if you are inclined, feel free to read the source code for the EventMonitor class and experiment with this function in your own flow/script.

In [23]:
# dai_monitor.event_handler()

### handle_event

WARNING: It is highly recomended that you do not interact with this function. However if you so chose, this function could be overridden to define custom behavior for handling events

NOTE: expecting returns from functions called on a w3.eth.filter class

In [1]:
# dai_monitor.handle_event(event)

## Helpers

### get_events_df

Returns processed events as a DataFrame

In [17]:
events_df = dai_monitor.get_events_df()
events_df.head()

args  \
block_number log_index                                                      
8928226      19         {'src': '0xc73e0383F3Aff3215E6f04B0331D58CeCf0...   
             20         {'src': '0xc73e0383F3Aff3215E6f04B0331D58CeCf0...   
             21         {'src': '0xc73e0383F3Aff3215E6f04B0331D58CeCf0...   
             22         {'src': '0xc73e0383F3Aff3215E6f04B0331D58CeCf0...   
             23         {'src': '0xc73e0383F3Aff3215E6f04B0331D58CeCf0...   

                           event  \
block_number log_index             
8928226      19         Approval   
             20         Approval   
             21         Approval   
             22         Approval   
             23         Approval   

                                                              transaction  \
block_number log_index                                                      
8928226      19         0xd6ec009c0b8daacbe5ad5cc9647a5df910d910081de6...   
             20         0xd6ec009c0b8daacbe5ad5cc9647a5df910d910081de6...   
             21         0xd6ec009c0b8daacbe5ad5cc9647a5df910d910081de6...   
             22         0xd6ec009c0b8daacbe5ad5cc9647a5df910d910081de6...   
             23         0xd6ec009c0b8daacbe5ad5cc9647a5df910d910081de6...   

                        log_index  transaction_index  \
block_number log_index                                 
8928226      19                19                 29   
             20                20                 29   
             21                21                 29   
             22                22                 29   
             23                23                 29   

                                                           address  \
block_number log_index                                               
8928226      19         0x9f8F72aA9304c8B593d555F12eF6589cC3A579A2   
             20         0xf53AD2c6851052A81B42133467480961B2321C09   
             21         0x89d24A6b4CcB1B6fAA2625fE562bDD9a23260359   
             22         0x89d24A6b4CcB1B6fAA2625fE562bDD9a23260359   
             23         0xC02aaA39b223FE8D0A0e5C4F27eAD9083C756Cc2   

                        block_number  \
block_number log_index                 
8928226      19              8928226   
             20              8928226   
             21              8928226   
             22              8928226   
             23              8928226   

                                                                    block  
block_number log_index                                                     
8928226      19         0xe8312bebafc94658685d97a87f90da716c904b95c27c...  
             20         0xe8312bebafc94658685d97a87f90da716c904b95c27c...  
             21         0xe8312bebafc94658685d97a87f90da716c904b95c27c...  
             22         0xe8312bebafc94658685d97a87f90da716c904b95c27c...  
             23         0xe8312bebafc94658685d97a87f90da716c904b95c27c...

### get_events_list

Returns process events as a List

In [19]:
events_list = dai_monitor.get_events_list()
events_list[:2]

[{'args': {'src': '0xc73e0383F3Aff3215E6f04B0331D58CeCf0Ab849',
   'guy': '0x448a5065aeBB8E423F0896E6c5D525C040f59af3',
   'wad': 115792089237316195423570985008687907853269984665640564039457584007913129639935},
  'event': 'Approval',
  'transaction': '0xd6ec009c0b8daacbe5ad5cc9647a5df910d910081de6d5146b4f8fc31e66980e',
  'log_index': 19,
  'transaction_index': 29,
  'address': '0x9f8F72aA9304c8B593d555F12eF6589cC3A579A2',
  'block_number': 8928226,
  'block': '0xe8312bebafc94658685d97a87f90da716c904b95c27cd1a81ace5888c3c66887'},
 {'args': {'src': '0xc73e0383F3Aff3215E6f04B0331D58CeCf0Ab849',
   'guy': '0x448a5065aeBB8E423F0896E6c5D525C040f59af3',
   'wad': 115792089237316195423570985008687907853269984665640564039457584007913129639935},
  'event': 'Approval',
  'transaction': '0xd6ec009c0b8daacbe5ad5cc9647a5df910d910081de6d5146b4f8fc31e66980e',
  'log_index': 20,
  'transaction_index': 29,
  'address': '0xf53AD2c6851052A81B42133467480961B2321C09',
  'block_number': 8928226,
  'block': '

### get_queue_size

Returns the size of the event handler queue. This number represents the amounts of unprocessed events which will be processed so long as `EventMonitor.get_handler_status() == 'RUNNING'`

In [20]:
event_queue_size = dai_monitor.get_queue_size()
event_queue_size

4655

### get_contract_events

This function is used internally for mapping ethereum event topics (HEX) to human readable event names. It can be interesting to see the kinds of events for contracts used to initalize the `EventMonitor` class

In [19]:
contract_events = dai_monitor.get_contract_events()
contract_events

0x6B175474E89094C44Da98b954EedeAC495271d0F  \
                                        name   
0                                   Approval   
1                                    LogNote   
2                                   Transfer   

                                                  \
                                            text   
0              Approval(address,address,uint256)   
1  LogNote(bytes4,address,bytes32,bytes32,bytes)   
2              Transfer(address,address,uint256)   

                                                      \
                                              keccak   
0  0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2...   
1  0xd3d8bec38a91a5f4411247483bc030a174e77cda9c03...   
2  0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4...   

                                                      
                                              inputs  
0  [{'indexed': True, 'internalType': 'address', ...  
1  [{'indexed': True, 'internalType': 'bytes4', '...  
2  [{'indexed': True, 'internalType': 'address', ...